In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import json
import time
import single_node_profiles as snp
import profiler
import end_to_end_profiles as e2e_profs
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline


In [3]:
profs = snp.load_single_node_profiles()

In [4]:
profs.keys()

dict_keys(['alexnet', 'res152', 'res18', 'res50', 'inception', 'tf-kernel-svm', 'tf-lang-detect', 'tf-log-reg', 'tf-lstm', 'tf-nmt', 'tf-resnet-feats'])

In [14]:
dag = profiler.get_logical_pipeline("pipeline_three")
with open(os.path.abspath("../results/e2e_profs/systemx/resnet_cascade/slo_500ms/alex_1-r50_1-r152_2-171025_083730.json")) as f:
    sample_run = json.load(f)
scale_factors = profiler.get_node_scale_factors(sample_run, dag.reference_node)
node_configs = profiler.get_node_configs_from_experiment(sample_run)
node_profs = {name : profiler.NodeProfile(profs[name]) for name, _ in node_configs.items()}

In [17]:
profiler.estimate_pipeline_performance_for_config(dag, scale_factors, node_configs, node_profs)

{'cost': 5.232499999999999,
 'latency': 0.4384363054107471,
 'throughput': 97.31799733443539}